In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import gc; gc.enable()

from utilities import DfLowMemory
from utilities import CleanData

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from category_encoders import WOEEncoder, TargetEncoder
from xgboost import XGBClassifier
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = DfLowMemory('train_ver2.csv')

In [ ]:
# ind_empleado - 27734. Employee index: A active, B ex employed, F filial, N not employee, P pasive. 
#I am noticing that there is a value that is not in the description so I believe it is a typo. 
#I tried looking more into the data but in the end I decided to drop since it is just one person.

df = df[df['ind_empleado'] != 'S']

In [ ]:
# pais_residencia - 27734. One aspect that I found interesting is that customers with pais_residenciamissing had all other features missing. 
#For now I am going to drop all of them

df.dropna(subset=['pais_residencia'], inplace=True)

In [ ]:
# Now there are 70 missing values for gender. The value distribution doesn't seem very different. I am going to replace them with the mode.
df['sexo'].fillna(df['sexo'].mode()[0], inplace=True)

In [ ]:
# ult_fec_cli_1t has a lot of missing values. This represents the last day the the customer was the primary costumer.
# Looking at the indrel_1mes column, it shows that most of the customers at the beginning of the month were the primary costumers. So 
# I am assuming that the customers with missing last date as the primary costumers are still the primary costumers. So I am going to impute 
# this with 'primary'
# THIS NEEDS TO BE MODIFIED TO TYPE DATE
df.loc[df['ult_fec_cli_1t'].isnull(), 'ult_fec_cli_1t'] = 'PRIMARY'

In [ ]:
# Customer type at the beginning of the month ,1 (First/Primary customer), 2 (co-owner ),P (Potential),3 (former primary), 4(former co-owner).
# some values that were supposed to be 1 are 1.0 and for the other categories as well. So I am going to use 1,2,3,4 like in the description
# This has been suggested by @StephenSmith
map_dict = {'1.0' : '1',
            '1' : '1',
            '2' : '2',
            '2.0' : '2',
            '3' : '3',
            '3.0' : '3', 
            '4' : '4',
            '4.0' : '4'}

df.indrel_1mes.fillna('P', inplace=True)
df.indrel_1mes = df.indrel_1mes.apply(lambda x: map_dict.get(x,x))
df.indrel_1mes = df.indrel_1mes.astype('category')

In [ ]:
# Customer relation type at the beginning of the month, A (active), I (inactive), P (former customer),R (Potential)
# There are some described as N that doesn't fit into the column description.
# Taking a closer look at these 4 rows and comparing them with the ind_actividad: df[df['tiprel_1mes'] == 'N'].iloc[:,8:24]
# The ones that had ind_actividad = 1 were imputed as active and the ones that had 0 were imputed as I
# For the nan values, there is another column named 'ind_actividad_cliente' that is 1 for active costumers and 0 for inactive. I am 
# going to use this to impute the nan values for tiprel_1mes

df.loc[df['tiprel_1mes'].isnull(), 'tiprel_1mes'] = df['ind_actividad_cliente']

map_tip = {1 : 'A',
           0 : 'I'}        

df.tiprel_1mes = df.tiprel_1mes.apply(lambda x: map_tip.get(x,x))
df.loc[6603017, 'tiprel_1mes'] = 'A'
df.loc[10123924, 'tiprel_1mes'] = 'A'
df.loc[10124648, 'tiprel_1mes'] = 'I'
df.loc[11247349, 'tiprel_1mes'] = 'I'

df.tiprel_1mes = df.tiprel_1mes.astype('category')

In [ ]:
# conyuemp Spouse index. 1 if the customer is spouse of an employee
# I am assuming that most customers are not spouses and I am going to impute these with the mode
# Now there are 70 missing values for gender. The value distribution doesn't seem very different. I am going to replace them with the mode.
df['conyuemp'].fillna(df['conyuemp'].mode()[0], inplace=True)
df.conyuemp = df.conyuemp.astype('category')

In [ ]:
# canal_entrada. Channel used by the customer to join
# I am going to fill the missing values with the mode
df['canal_entrada'].fillna(df['canal_entrada'].mode()[0], inplace=True)

In [ ]:
# tipodom. Addres type. 1, primary address. It is only 1 value missing so I am just going to replace it with the mode
df['tipodom'].fillna(df['tipodom'].mode()[0], inplace=True)

In [ ]:
# cod_prov. Province code (customer's address)
# nomprov. Province name. 
# These two have the same number of missing values. I am going to replace these with unknow
# THESE IS SUGGESTED BY ALAN PRYOR

for c in ['cod_prov', 'nomprov']:
    df.loc[df[c].isnull(), c ] = 'UNKOWN'

In [ ]:
# Renta. Gross income of the household. I am going to replace the missing values with the mean salary per providence. 

salaries = dict(df.groupby('nomprov')['renta'].mean().round(0))
df.loc[df['renta'].isnull(), 'renta'] = df['nomprov']
df.renta = df.renta.apply(lambda x: salaries.get(x,x))

In [ ]:
# Segmento. segmentation: 01 - VIP, 02 - Individuals 03 - college graduated
for c in ['segmento']:
    df.loc[df[c].isnull(), c ] = 'UNKOWN'

In [ ]:
df['ind_nomina_ult1'].fillna(df['ind_nomina_ult1'].mode()[0], inplace=True)

In [ ]:
df['ind_nom_pens_ult1'].fillna(df['ind_nom_pens_ult1'].mode()[0], inplace=True)

In [ ]:
df['fecha_dato'] = pd.to_datetime(df['fecha_dato'], format="%Y-%m-%d")
df['fecha_alta'] = pd.to_datetime(df['fecha_alta'], format="%Y-%m-%d")

In [ ]:
df.loc[df['antiguedad'] == '-999999', 'antiguedad'] = 'UNKNOWN'

In [ ]:
df['fecha_dato'].unique()

# Missing values and action plan

ind_empleado - 27734. Employee index: A active, B ex employed, F filial, N not employee, P pasive. I am noticing that there is a value that is not in the description so I believe it is a typo. I tried looking more into the data but in the end I decided to drop since it is just one person. The nan values I am replacing with the mode which in this case it is that the person is not an employee.

df = df[df['ind_empleado'] != 'S']

-------------------------------------------------------
pais_residencia - 27734. One aspect that I found interesting is that customers with pais_residenciamissing had all other features missing. For now I am going to drop all of them

filtered_df = df[df['pais_residencia'].isnull()]
len(filtered_df['ncodpers'].unique())
This shows that there are 7340 without much information about them

-------------------------------------------------------
sexo - 27804. Most of the clients that have this information missing dont have much information on them overall. If we were to drop the ones that have most information missing, sexo would have about 70 values missing. For now I am going to change these with the mode.

-------------------------------------------------------
I need to turn fecha_dato and fecha_alta into dates

fecha_alta - 27734. This is the date that the customer joined the bank.
Added another column called month. May be useful in the case that some customers buy more depending on the time of the year.

-------------------------------------------------------
ult_fec_cli_1t - 13622516. This is the last date as a primary customer.
indrel_1mes                149781
tiprel_1mes                149781
indresi                     27734
indext                      27734
conyuemp                 13645501
canal_entrada              186126
indfall                     27734
tipodom                     27735
cod_prov                    93591
nomprov                     93591
ind_actividad_cliente       27734
renta                     2794375
segmento                   189368
ind_nomina_ult1             16063
ind_nom_pens_ult1           16063


-------------------------------------------------------
A bank guarantee is a lending institution's promise to cover a loss if a borrower (their customer) defaults on a debt to a third party. 

A payroll account is a separate bank account for your business that is strictly used for payroll. Instead of lumping all your business expenses into one account, you will pay employee wages with your payroll bank account. The money going into the payroll account will only be used for payroll.

Bank policy requires that account owners be the legal age of 18 in order to open an account online. If you are not 18 years old, you may open an account for minors with your parent or legal guardian as the joint account holder.

A pension plan is a type of retirement plan where an employee adds money into a fund that includes contributions by the employer.

A securities account sometimes known as a brokerage account is an account that holds financial assets such as securities on behalf of an investor with a bank, broker or custodian.

In [ ]:
used_cols = [c for c in df.columns.tolist() if c not in ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']]

## Feature Engineering

In [ ]:
# Like 1st place solution mentioned, some features worth exploring are lags of products time since presence of products, average of products, time since last purchase
# of products
# time since change and lads for a non-product attributes: segmento, indactividadclient, codprov, canalentrada, indrel1mes, tiprel1mes

In [ ]:
frames = []
for dato in df['fecha_dato'].unique().tolist():
    # get 10% sample for each day
    df_dato = df.loc[df['fecha_dato'] == dato].copy().sample(frac=0.10)

    # add to frames list
    frames.append(df_dato)
    gc.collect()


In [ ]:
trial = pd.concat(frames)

In [ ]:
trial['fecha_dato'] = pd.to_datetime(trial['fecha_dato'], format="%Y-%m-%d")
trial['fecha_alta'] = pd.to_datetime(trial['fecha_alta'], format="%Y-%m-%d")

In [ ]:
sns.barplot(x= trial['fecha_dato'], y=trial['ind_ahor_fin_ult1'])

In [ ]:
trial['month'] = trial['fecha_dato'].dt.month

In [ ]:
trial['YearMonth'] = trial['fecha_dato'].map(lambda x: 100*x.year + x.month)

In [ ]:
df['fecha_dato'] = pd.to_datetime(df['fecha_dato'], format="%Y-%m-%d")
df['fecha_alta'] = pd.to_datetime(df['fecha_alta'], format="%Y-%m-%d")

In [ ]:
df['YearMonth'] = df['fecha_dato'].map(lambda x: 100*x.year + x.month)

In [ ]:
plt.figure(figsize=(16, 6))
sns.barplot(x= df['YearMonth'], y = df['ncodpers'])

In [ ]:
trial['JoinYear'] = trial['fecha_alta'].apply(lambda x: 100*x.year + x.month)

In [ ]:
d = trial.groupby('ncodpers')

In [ ]:
plt.figure(figsize=(16, 6))
sns.barplot(x= df['JoinYear'], y = df['ncodpers'])

In [ ]:
new_customer = list(set(trial['ncodpers']))

In [ ]:
new_cust_join_date = []

In [ ]:
for customer in new_customer:
    new_customer_join_date.append(trial['fecha_alta'])

In [ ]:
unique_months = pd.DataFrame(pd.Series(trial.fecha_dato.unique())).reset_index(drop=True)

In [ ]:
unique_months["month_id"] = pd.Series(range(1,1+unique_months.size))

In [ ]:
unique_months.head()

In [ ]:
unique_months["month_next_id"] = 1 + unique_months["month_id"]

In [ ]:
unique_months.head()

In [ ]:
unique_months.rename(columns={0:"fecha_dato"},inplace=True)

In [ ]:
unique_months.head()

## Building a recommendation system

For a recommendation system first we need a distance metric so that we can find users who are similar to a given user.

In [ ]:
target = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [ ]:
used_cols = [c for c in trial.columns.tolist() if c not in [target, 'fecha_dato', 'fecha_alta']]

In [ ]:
X = trial[used_cols].values
y = trial[target]

In [ ]:
encoder = TargetEncoder()

for c in trial[ohe]:
    print(c)
    print(trial[c].unique())
    print('\n')

for t in target:
    X = trial[used_cols]
    y = trial[t]
    
    encoder = TargetEncoder()
    X = encoder.fit_transform(X,y)
    
    model = X
    

In [ ]:
X = trial[used_cols].values
y = trial['ind_ahor_fin_ult1']

In [ ]:
X = encoder.fit_transform(X,y)

In [ ]:
# train test split

train_size = int(len(X) * 0.7)
y_train = int(len(y) * 0.7)
X_train, X_test = X[0:train_size], X[train_size:len(X)]
y_train, y_test = y[0:train_size], y[train_size:len(X)]
print('Observations: ', (len(X)))
print('Training Observations: ', (len(train)))
print('Testing Observations: ', (len(test)))

In [ ]:
model = imb_xgb(special_objective='weighted', imbalance_alpha=2)

In [ ]:
model.fit(X_train.values, y_train.values)

In [ ]:
y_pred = model.predict(X_test.values)

In [ ]:
### Building the 24 models

predictions = []
models = []
encoder = TargetEncoder()

for t in target:
    X = trial[used_cols].values
    y = trial[t]
    
    X = encoder.fit_transform(X,y)
    
    train_size = int(len(X) * 0.7)
    y_train = int(len(y) * 0.7)
    X_train, X_test = X[0:train_size], X[train_size:len(X)]
    y_train, y_test = y[0:train_size], y[train_size:len(X)]
    
    model = imb_xgb(special_objective='weighted', imbalance_alpha=2)
    model.fit(X_train.values, y_train.values)
    models.append(model)
    
    y_pred = model.predict(X_test.values)
    predictions.append(y_pred)

In [ ]:
zero_one_predictions = []
for y_pred in predictions:
    y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())
    y_pred.min(), y_pred.max()
    zero_one_predictions.append(y_pred)

In [ ]:
test_id = np.array(trial[train_size:]['ncodpers'])

In [ ]:
preds = np.argsort(zero_one_predictions, axis=1)

In [ ]:
preds

In [ ]:
preds = np.fliplr(preds)[:,:7]

In [ ]:
preds

In [ ]:
test_id = np.array(pd.read_csv('test_ver2.csv', usecols=['ncodpers'])['ncodpers'])

In [ ]:
final_preds = []

for i, pred in enumerate(preds):
    
    cust = test_id[i]
    top_products = target[pred]
    used_products = customer_dict.get(cust,[])
    
    new_top_products = []
    
    for product in top_products: 
        if product not in used_products:
            new_top_products.append(product)
        if len(new_top_products) == 7:
            break
    final_preds.append(" ".join(new_top_products))

In [ ]:
submission = pd.DataFrame({'ncodpers': test_id, 'added_products': final_preds})
submission.to_csv(kaggle_submission, index=False)

# Lags

Lagging of independent variables is often necessary in order for the regression model to be able to predict the future--i.e., to predict what will happen in period t based on knowledge of what happened up to period t-1

## Modeling without train test split

In [ ]:
target = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [ ]:
used_cols = [c for c in trial.columns.tolist() if c not in [target, 'fecha_dato', 'fecha_alta']]

In [ ]:
df_test = pd.read_csv('test_ver2.csv')

In [ ]:
df_test.head(2)

In [ ]:
predictions = []
models = []
encoder = TargetEncoder()

for t in target:
    X = trial[used_cols].values
    y = trial[t]
    
    X = encoder.fit_transform(X,y)
    
    model = imb_xgb(special_objective='weighted', imbalance_alpha=2)
    model.fit(X.values, y.values)
    models.append(model)
    
    y_pred = model.predict(X_test.values)
    predictions.append(y_pred)

In [ ]:
precision_score(trial[train_size:][target].values, np.array(zero_one_predictions).T, average='macro')

In [ ]:
test_df = pd.read_csv('test_ver2.csv')

In [ ]:
test_df.head(2)

In [ ]:
test_id = X_test[0]

In [ ]:
zero_one_predictions[1]

In [ ]:
to_drop = [c for c in df.columns.tolist() if c not in target + ['ncodpers']]

In [ ]:
customer_id = trial.drop(columns=to_drop)

In [ ]:
customer_id = customer_id.drop_duplicates('ncodpers', keep='last')

In [ ]:
customer_dict = {}
target = np.array(target)

In [ ]:
customer_id.set_index('ncodpers', inplace=True)
customer_id.head()

In [ ]:
for c in customer_id.columns.tolist():
    customer_id[c] = customer_id[c].astype(bool)
    
customer_id.head()

In [ ]:
for i, row in customer_id.iterrows():
    cust = row['ncodpers']
    used_products = set(target[np.array(row[1:])==1])
    customer_dict[cust] = used_products

In [ ]:
zero_one_predictions

In [ ]:
from sklearn.metrics import precision_score, recall_score

In [ ]:
precision_score(y_test, y_pred > 0.5)

In [ ]:
recall_score(y_test, y_pred > 0.5)

In [ ]:
def _apk(y_test, y_pred, k=7):
    
    # adjust the length if necessary
    if len(y_pred) > k:
        y_pred = y_pred[:k]
        
    score = 0.0
    num_hits = 0.0
    
    for i, p in enumerate(y_pred):
        if p in y_test and p not in y_pred[:i]:
            num_hits += num_hits / (i+1.0)
            
        if not y_test:
            return 0.0
        
    return score / min(len(y_test), k)

In [ ]:
def mapk (y_test,y_pred, k=7):
    
    return np.mean([_apk(a,p,k) for a,p in zip(y_test, y_pred)])

In [ ]:
print(mapk(y_test, zero_one_predictions))

In [ ]:
print(mapk(y_test, y_pred))

In [ ]:
# Need to find the products that customers do not have
# if user has every product nothing should be recommended

In [ ]:
balanced_accuracy_score(y_test, y_pred > 0)

In [ ]:
roc_auc_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred > 0)

In [ ]:
print(classification_report(y_test, y_pred > 0))

In [ ]:
# Second target
X = trial[used_cols].values
y = trial['ind_aval_fin_ult1']

In [ ]:
encoder = TargetEncoder()
X = encoder.fit_transform(X,y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model2 = imb_xgb(special_objective='weighted', imbalance_alpha=2)

In [ ]:
model2.fit(X_train.values, y_train.values)

In [ ]:
y_pred2 = model2.predict(X_test.values)

In [ ]:
print(classification_report(y_test, y_pred2 > 0))

In [ ]:
### Building the 24 models

predictions = []
models = []
encoder = TargetEncoder()

for t in target:
    X = trial[used_cols].values
    y = trial[t]
    
    X = encoder.fit_transform(X,y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    model = imb_xgb(special_objective='weighted', imbalance_alpha=2)
    model.fit(X_train.values, y_train.values)
    models.append(model)
    
    y_pred = model.predict(X_test.values)
    predictions.append(y_pred)

In [ ]:
for i, row in customer_id.iterrows():
    cust = row['ncodpers']
    used_products = set(target[np.array(row[1:])==1])
    customer_dict[cust] = used_products

In [ ]:
	print("Getting the top products..")
	target_cols = np.array(target_cols)
	preds = np.argsort(preds, axis=1)
	preds = np.fliplr(preds)[:,:7]
	test_id = np.array(pd.read_csv(data_path + "test_ver2.csv", usecols=['ncodpers'])['ncodpers'])
	final_preds = [" ".join(list(target_cols[pred])) for pred in preds]
	out_df = pd.DataFrame({'ncodpers':test_id, 'added_products':final_preds})
	out_df.to_csv('sub_xgb_new.csv', index=False)
	print(datetime.datetime.now()-start_time)